In [96]:
from dotenv import dotenv_values
config = dotenv_values("../.env")

import os
import pandas as pd
from datetime import datetime
import time

from riotwatcher import LolWatcher, ApiError

""" 
API Rate Limits:
20 requests every 1 seconds(s)
100 requests every 2 minutes(s)
"""

watcher = LolWatcher(config['riotwatcher_api_key'])

In [97]:
# record start time
start_timer = time.time()

""" user-defined variables """
var_region = "euw1"
var_summoner_name= "AdAstraData"

""" fetch item_list from data_dragon and create dict with item_id : item_name """
data_dragon_versions = watcher.data_dragon.versions_for_region(region=var_region)
data_dragon_item_version = data_dragon_versions['n']['item']

item_list = watcher.data_dragon.items(data_dragon_item_version)

dict_item_id_item_name = {}

for item_id in item_list['data'].keys():
    df_item_id = pd.json_normalize(data=item_list['data'][item_id])
    item_name = df_item_id['name'].to_string(index=False)
    dict_item_id_item_name[item_id] = item_name

""" store summoner ids """
summoner_puuid = watcher.summoner.by_name(region=var_region, summoner_name=var_summoner_name)
my_puuid = summoner_puuid['puuid']

""" find all matches played by puuid """
last_matches_puuid = watcher.match.matchlist_by_puuid(
    region=var_region, 
    puuid=my_puuid, 
    count=100
    )

# record end time
end_timer = time.time()

# number of API calls
num_api_calls = 4

print("Execution time:",(end_timer-start_timer), "s")
print("Number of API calls:",num_api_calls)
print("API calls per second:",round(num_api_calls/(end_timer-start_timer),2))

Execution time: 1.4049055576324463 s
Number of API calls: 4
API calls per second: 2.85


In [98]:
""" list of match_id for all matches played by summoner"""
len(last_matches_puuid)

100

In [127]:
dict_match_info = {
    "match_id": [],
    "team_id": [],
    "participant_puuid": [],
    "summoner_name": [],
    "gold_earned": [],
}

""" for each match_id """
for var_match_id in last_matches_puuid[0:1]:

    """ GET summoner puuids """
    watcher_match = watcher.match.by_id(region=var_region, match_id=var_match_id)

    for puuid_num in range(len(watcher_match['metadata']['participants'])):
        puuid_features = watcher_match['info']['participants'][puuid_num]

        dict_match_info['match_id'].append(var_match_id)
        dict_match_info['team_id'].append(puuid_features['teamId'])
        dict_match_info['participant_puuid'].append(puuid_features['puuid'])
        dict_match_info['summoner_name'].append(puuid_features['summonerName'])
        dict_match_info['gold_earned'].append(puuid_features['goldEarned'])
    
    """ time delay with time.sleep """
    time.sleep(0.5)


In [128]:
df_match_info = pd.DataFrame.from_dict(dict_match_info)
df_match_info['team_id'] = ['Blue' if team_id == 100 else 'Red' for team_id in df_match_info['team_id']]
df_match_info

,match_id,team_id,participant_puuid,summoner_name,gold_earned
0,EUW1_6162024635,Blue,pdmWhZjkPNzJ2FIcmd5YfkxyNmFTCVbMbL2WOaWgMdck2z...,Storm and Sin,6124
1,EUW1_6162024635,Blue,bAbkdqqlpnuDlKFo32I_PBfi-yO2R8eBcD4qo9hqkAZ8Nm...,Ariiial,4539
2,EUW1_6162024635,Blue,bZZNu22YOLpn8iDq1FCmW2u4fjW6hNGed3YKTh9LvQeNI6...,AdAstraData,6727
3,EUW1_6162024635,Blue,9fZyZmC705BV1nflyxYDE_qtc4EnxO0SsCjIZ6DG2PV-Fi...,Glockenhorst,9177
4,EUW1_6162024635,Blue,wVANHowswD8AnQLqwxf6j6g_6XHjf7lwyWuN3VNnHHJ3jR...,Franky221b,3621
5,EUW1_6162024635,Red,6CeQqFGkUSp1sqOxUZcH7hND37eFr48vciL_kKH3X1Kzsa...,krandor305,7466
6,EUW1_6162024635,Red,CH6hjQ8KEhGavJTOonNqLyRw38G0iTIeb4GEPKz1oIXlSY...,smovii,10202
7,EUW1_6162024635,Red,vIjjJw3_YiKmTiCeFgF5EhDXnv5YsvjvFEJx_9q9jbr9OM...,Tugas19,6414
8,EUW1_6162024635,Red,xFrmSIU9Fqjp68kmFyy5NJTyrdyrSQxsWeyYmO6fawEGr0...,Jinn1887,6927
9,EUW1_6162024635,Red,c_Pi11J6hRSS01Hj3Fz84Qh-2lB7UdlD01mhnD_g67Lv1d...,fluffy333,4335


In [143]:
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite:///riotwatcher_api_dt.db', echo=False)

for var_match_id in df_match_info['match_id'].unique():
    df_tmp = pd.read_sql_query("SELECT * from tbl_match_info WHERE match_id = '%s'" %(var_match_id), con=engine)
    
    if df_tmp.empty:
        df_append = df_match_info.query("match_id == @var_match_id")
        df_append.to_sql('tbl_match_info', con=engine, if_exists='append', index=False)
    else:
        pass

In [145]:
df = pd.read_sql_query("SELECT * from tbl_match_info", con=engine)
df

,match_id,team_id,participant_puuid,summoner_name,gold_earned
0,EUW1_6162024635,Blue,pdmWhZjkPNzJ2FIcmd5YfkxyNmFTCVbMbL2WOaWgMdck2z...,Storm and Sin,6124
1,EUW1_6162024635,Blue,bAbkdqqlpnuDlKFo32I_PBfi-yO2R8eBcD4qo9hqkAZ8Nm...,Ariiial,4539
2,EUW1_6162024635,Blue,bZZNu22YOLpn8iDq1FCmW2u4fjW6hNGed3YKTh9LvQeNI6...,AdAstraData,6727
3,EUW1_6162024635,Blue,9fZyZmC705BV1nflyxYDE_qtc4EnxO0SsCjIZ6DG2PV-Fi...,Glockenhorst,9177
4,EUW1_6162024635,Blue,wVANHowswD8AnQLqwxf6j6g_6XHjf7lwyWuN3VNnHHJ3jR...,Franky221b,3621
...,...,...,...,...,...
995,EUW1_6132387357,Red,60eathQMCVHt8cz85AlWokhEnFhfH45Mvsy1eUtk9EY9an...,YNSliberable,6308
996,EUW1_6132387357,Red,LL9AwcYm05NM0COm9pBSMGaQq1F3zojOQ6SGRzLpNc8ccw...,shiuliberable,7248
997,EUW1_6132387357,Red,tRB7vNjqLtZcgdsFFU-A4f0fgxrnfSorK_le8zp5l3Sp4r...,LlunaYuzu,4693
998,EUW1_6132387357,Red,BsETsdRb4N35gEvjEXZWKcyQYAQXbocv208a7f6eetsHbG...,AS Kenox,4250


In [ ]:
### ACTION_ITEMS : 
# 4. Add new data and test if append respects existing match info

In [4]:
list_match_item_purchased = []

for var_match_id in last_matches_puuid[0:10]:

    """ store participant's puuids """
    dict_participants_puuids = {}

    match_participants = watcher.match.by_id(region=var_region, match_id=var_match_id)
    list_match_participants = match_participants['metadata']['participants']
    
    for idx in range(len(list_match_participants)):
        participant_id = idx + 1
        dict_participants_puuids[str(participant_id)] = list_match_participants[idx]

    """ fetch match_timeline """
    match_info = watcher.match.timeline_by_match(region=var_region,match_id=var_match_id)
    match_timeline = pd.json_normalize(data=match_info['info']['frames'], record_path='events')

    """ add match_id to match_timeline"""
    match_timeline['match_id'] = var_match_id

    """ convert match_timeline' timestamp to elapsed minutes"""
    match_timeline['timestamp_timedelta'] = pd.to_timedelta(match_timeline['timestamp'], unit='ms')
    match_timeline['elapsed_minutes'] = round(match_timeline['timestamp_timedelta']/ pd.Timedelta(minutes=1),3)
    
    """ subset for item_purchased """
    item_purchased = match_timeline.query("type == 'ITEM_PURCHASED'").dropna(axis=1)

    """ retrieve item_name """
    item_purchased['item_id'] = item_purchased['itemId'].astype(int)
    item_purchased['item_name'] = [dict_item_id_item_name[str(item)] for item in item_purchased['item_id']]

    """ retrieve summoner_puuid """
    item_purchased['participant_id'] = item_purchased['participantId'].astype(int)
    item_purchased['summoner_puuid'] = [dict_participants_puuids[str(id)] for id in item_purchased['participant_id']]

    """ subset for @my_puuid """
    item_purchased = item_purchased.query("summoner_puuid == @my_puuid")

    """ select cols - elapsed_minutes, item_name, summoner_puuid """
    item_purchased = item_purchased[
        [
            'match_id', 
            'elapsed_minutes',
            'item_name',
            'summoner_puuid'
        ]
    ]
    
    """ create dict with item_purchased per timestamp per participant (summoner) """
    dict_item_purchased = item_purchased.to_dict(orient='records')
    list_match_item_purchased.append(dict_item_purchased)

# record end time
end_timer = time.time()

# number of API calls
num_api_calls_per_cycle = 1
num_api_cycles = len(last_matches_puuid)
num_api_calls = num_api_cycles * num_api_calls_per_cycle

print("Execution time:",(end_timer-start_timer), "s")
print("Number of API calls:",num_api_calls)
print("API calls per second:",round(num_api_calls/(end_timer-start_timer),2))

Execution time: 14.254740476608276 s
Number of API calls: 100
API calls per second: 7.02


In [5]:
""" obtain avg. elapsed_minutes per item_name (last matches)"""
from itertools import chain

df_match_item_purchased = pd.DataFrame(list(chain.from_iterable(list_match_item_purchased)))

avg_min_Dorans_Ring = df_match_item_purchased.query('item_name == "Doran\'s Ring"')['elapsed_minutes'].describe()
avg_min_Dorans_Ring

count    24.000000
mean      0.259708
std       0.146596
min       0.097000
25%       0.155500
50%       0.209000
75%       0.322000
max       0.614000
Name: elapsed_minutes, dtype: float64

In [ ]:
### ACTION_ITEMS : 
# 1. need to test first for API call before proceeding (hitting the API's rate limits as it is) 
# 2. need to add order_num to item_name to distinguish between similar item_names bought at different timestamps
# 3. .describe() to be applied per item_name_order_num 